In [ ]:
import pandas as pd

In [ ]:
log = pd.read_csv('visit_log.csv', sep = ';')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


Задание 1
Для датафрейма log из материалов занятия создайте столбец source_type по правилам:

если источник traffic_source равен Yandex или Google, то в source_type ставится organic;
для источников paid и email из России ставим ad;
для источников paid и email не из России ставим other;
все остальные варианты берём из traffic_source без изменений.



In [ ]:
#1

In [ ]:
log.loc[log.traffic_source.str.contains('email|google'),'source_type'] = 'organic'#я бы хотел реализовать решение с помощью новых медтодов
#но не разобрался как сформулировать в одном локе разные условия с разным выводом
log.head(25)

In [ ]:
def source(row):#я так понял, что ни isin, ни str.contains не применимы к строке, поэтому пришлось вот так подробно прописывать все условия

  if row['traffic_source'] == ('yandex') or row['traffic_source'] == ('google'):
    row['source_type'] = 'organic'
  elif (row['traffic_source'] == ('paid') or row['traffic_source'] == ('email')) and (row['region'] == ('Russia')):
    row['source_type'] = 'ad'
  elif (row['traffic_source'] == ('paid') or row['traffic_source'] == ('email')) and (row['region'] != ('Russia')):
    row['source_type'] = 'other'
  else:
    row['source_type'] = row['traffic_source']

  return row

In [ ]:
log.apply(source, axis = 1).head(20)

In [ ]:
#2

В файле URLs.txt содержатся URL страниц новостного сайта. Вам нужно отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри URL конструкцию: /, затем 8 цифр, затем дефис. Выполните действия:

Прочитайте содержимое файла с датафрейм.
Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствие с заданным шаблоном.


In [ ]:
urls = pd.read_csv('URLs.txt', sep = '\n')
urls.head()

,url
0,/world/
1,/latest/
2,/?updated=top
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [ ]:
urls[urls.url.str.contains('\d{8}-')].head()

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [ ]:
#3
ratings = pd.read_csv('ratings.csv')
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значениями столбца timestamp для данного значения userId.



In [ ]:
grouped = ratings.groupby('userId').agg({'rating':'count'}).reset_index()#не понимаю почему здесь не сработал фильтр с группировкой 
#.filter(lambda x: x['rating'] > 100). В общем, посчтитал в группировке по айди количество оценок 
grouped.head()

,userId,rating
0,1,20
1,2,76
2,3,51
3,4,204
4,5,100


In [ ]:
grouped.query('rating > 100')#258 пользователей 100+ оценок. Тут тоже не понятно, почему обычная фильтрация через обращение к колонке или 
#через loc не срабатывает: ошибка unhashable type 'Series'. Query какой-то появляется

,userId,rating
3,4,204
7,8,116
14,15,1700
16,17,363
18,19,423
...,...,...
655,656,128
658,659,142
663,664,519
664,665,434


In [ ]:
df_merged = grouped.query('rating > 100').merge(ratings, left_on='userId', right_on='userId')#после объединения оказалось, что эти 258 пользователей
#со 100+ оценками в сумме наставили 80487 оценок.
df_merged

,userId,rating_x,movieId,rating_y,timestamp
0,4,204,10,4.0,949810645
1,4,204,34,5.0,949919556
2,4,204,112,5.0,949810582
3,4,204,141,5.0,949919681
4,4,204,153,4.0,949811346
...,...,...,...,...,...
80482,671,115,6268,2.5,1065579370
80483,671,115,6269,4.0,1065149201
80484,671,115,6365,4.0,1070940363
80485,671,115,6385,2.5,1070979663


In [ ]:
df_life = df_merged.groupby('userId').agg({'timestamp':['min','max']})#опять группировка, по каждому айди получил его мин и макс таймстэмп
df_life.head()

timestamp            
               min         max
userId                        
4        949778714   949982274
8       1154389340  1154474527
15       997937239  1469330735
17      1127468587  1127476640
19       855190091   855195373

In [ ]:
def average_life(row):#функция, которая считает время жизни в секундах для каждого пользователя
  return row['timestamp', 'max'] - row['timestamp', 'min']

In [ ]:
#после применения функции все те же 258 строк время жизни каждого пользователя в секундах
#вопрос: как это вычислить в виде lambda? я очень хотел, но не смог
#потом к результату функции применяется метод mean(), получаю среднее время для всех
#и в самую последнюю очередь timedelta переводит секунды в дни
pd.Timedelta(df_life.apply(average_life, axis =1).mean(),unit='s')#все те же 258 строк время жизни пользователя в секундах

Timedelta('463 days 21:28:27.449612401')

In [ ]:
import sqlite3

In [ ]:
con = sqlite3.connect('ratings.db')

In [ ]:
ratings.to_sql('ratings', con, if_exists='replace')

In [ ]:

pd.read_sql('select avg(life_t) from\
                (select userID, count(rating) as num_r, \
                max(timestamp)-min(timestamp) as life_t \
                from ratings group by 1) t\
             where num_r > 100', con)

#вот такая реализация получилась методами SQL. C ходу прям не могу вспомнить как эти секунды в нормальное восприятие 
#я проверил в ДБивере сопоставимое значение получилось. Секунды:минуты: часы совпадают, расхождение в неск дней, возможно я экпоненциальный формат 
#неправильно перевел

,avg(life_t)
0,4.008051e+07


In [ ]:
value=str('4.008051e+07')
value2=value.replace(',', '.')
pd.Timedelta(float(value2), unit = 's')


Timedelta('463 days 21:28:30')

In [ ]:
#4

Дана статистика услуг перевозок клиентов компании по типам (см. файл “Python_13_join.ipynb” в разделе «Материалы для лекции “Продвинутый pandas”» ---- Ноутбуки к лекции «Продвинутый pandas»).
Нужно сформировать две таблицы:

таблицу с тремя типами выручки для каждого client_id без указания адреса клиента;
аналогичную таблицу по типам выручки с указанием адреса клиента.

In [ ]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [ ]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [ ]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [ ]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [ ]:
from functools import reduce

In [ ]:
df_reduced = reduce(lambda a, b: a.merge(b, how='outer', on='client_id'), [rzd, auto, air])
df_reduced

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [ ]:
df_adress = df_reduced.merge(client_base, on = 'client_id', how = 'left')
df_adress

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4
